# Analyzing movie review sentiment with RNNs

## Analyzing and preprocessing the data

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras import layers, models, losses, optimizers
from tensorflow.keras.preprocessing.sequence import  pad_sequences

In [25]:
vocab_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x00000190CB0FED40>
Traceback (most recent call last):
  File "c:\Users\jko2\Anaconda3\envs\test\lib\weakref.py", line 371, in remove
    self = selfref()
KeyboardInterrupt: 


In [ ]:
print('Number of training samples:', len(y_train))
print('Number of positive samples', sum(y_train))
print('Number of test samples:', len(y_test))

print(X_train[0])

In [ ]:
word_index = imdb.get_word_index()
index_word = {index: word for word, index in word_index.items()}

print([index_word.get(i, ' ') for i in X_train[0]])

In [ ]:
review_lengths = [len(x) for x in X_train]
import matplotlib.pyplot as plt
plt.hist(review_lengths, bins=10)
plt.show()


In [ ]:
maxlen = 200

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
print('X_train shape after padding:', X_train.shape)
print('X_test shape after padding:', X_test.shape)


## Building a simple LSTM network

In [ ]:


tf.random.set_seed(42)

model = models.Sequential()
embedding_size = 32
model.add(layers.Embedding(vocab_size, embedding_size))
model.add(layers.LSTM(50))
model.add(layers.Dense(1, activation='sigmoid'))

print(model.summary())

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


batch_size = 64
n_epoch = 3

In [ ]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=n_epoch,
          validation_data=(X_test, y_test))

acc = model.evaluate(X_test, y_test, verbose = 0)[1]

print('Test accuracy:', acc)


# Stacking multiple LSTM layers

In [ ]:
model = models.Sequential()
model.add(layers.Embedding(vocab_size, embedding_size))
model.add(layers.LSTM(50, return_sequences=True, dropout=0.2))
model.add(layers.LSTM(50, dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))

print(model.summary())

optimizer = optimizers.Adam(lr=0.003)

model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

n_epoch = 7
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=n_epoch,
          validation_data=(X_test, y_test))

acc = model.evaluate(X_test, y_test, verbose=0)[1]

print('Test accuracy with stacked LSTM:', acc)


<hr>

# Writing your own War and Peace with RNNs

## Acquiring and analyzing the training data

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, losses, optimizers

import numpy as np

from tensorflow.keras.callbacks import Callback, ModelCheckpoint, EarlyStopping

In [ ]:

training_file = 'warpeace_input.txt'

raw_text = open(training_file, 'r').read()
raw_text = raw_text.lower()
print(raw_text[:200])

all_words = raw_text.split()
unique_words = list(set(all_words))
print(f'Number of unique words: {len(unique_words)}')

In [ ]:
n_chars = len(raw_text)
print(f'Total characters: {n_chars}')
chars = sorted(list(set(raw_text)))
n_vocab = len(chars)
print(f'Total vocabulary (unique characters): {n_vocab}')
print(chars)

## Constructing the training set for the RNN text generator

In [ ]:
index_to_char = dict((i, c) for i, c in enumerate(chars))
char_to_index = dict((c, i) for i, c in enumerate(chars))
print(char_to_index)


In [ ]:

seq_length = 160
n_seq = int(n_chars / seq_length)

X = np.zeros((n_seq, seq_length, n_vocab))
Y = np.zeros((n_seq, seq_length, n_vocab))

In [ ]:
for i in range(n_seq):
	x_sequence = raw_text[i * seq_length : (i + 1) * seq_length]
	x_sequence_ohe = np.zeros((seq_length, n_vocab))
	for j in range(seq_length):
		char = x_sequence[j]
		index = char_to_index[char]
		x_sequence_ohe[j][index] = 1.
	X[i] = x_sequence_ohe
	y_sequence = raw_text[i * seq_length + 1 : (i + 1) * seq_length + 1]
	y_sequence_ohe = np.zeros((seq_length, n_vocab))
	for j in range(seq_length):
		char = y_sequence[j]
		index = char_to_index[char]
		y_sequence_ohe[j][index] = 1.
	Y[i] = y_sequence_ohe

print(X.shape)
print(Y.shape)

## Building an RNN text generator

In [ ]:
batch_size = 20
hidden_units = 700
n_epoch= 300
dropout = 0.4

tf.random.set_seed(42)

model = models.Sequential()
model.add(layers.LSTM(hidden_units, input_shape=(None, n_vocab), return_sequences=True, dropout=dropout))
model.add(layers.LSTM(hidden_units, return_sequences=True, dropout=dropout))

model.add(layers.TimeDistributed(layers.Dense(n_vocab, activation='softmax')))

optimizer = optimizers.RMSprop(lr=0.001)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


print(model.summary())

## Training the RNN text generator

In [ ]:
def generate_text(model, gen_length, n_vocab, index_to_char):
    """
    Generating text using the RNN model
    @param model: current RNN model
    @param gen_length: number of characters we want to generate
    @param n_vocab: number of unique characters
    @param index_to_char: index to character mapping
    @return:
    """
    # Start with a randomly picked character
    index = np.random.randint(n_vocab)
    y_char = [index_to_char[index]]
    X = np.zeros((1, gen_length, n_vocab))
    for i in range(gen_length):
        X[0, i, index] = 1.
        indices = np.argmax(model.predict(X[:, max(0, i - 99):i + 1, :])[0], 1)
        index = indices[-1]
        y_char.append(index_to_char[index])
    return ''.join(y_char)


class ResultChecker(Callback):
    def __init__(self, model, N, gen_length):
        self.model = model
        self.N = N
        self.gen_length = gen_length

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.N == 0:
            result = generate_text(self.model, self.gen_length, n_vocab, index_to_char)
            print('\nMy War and Peace:\n' + result)



Please note: the next code cell will take a long time to run. you can try running this on google colab or let it run in the background whilst you work on other tasks.

In [ ]:
filepath="weights/weights_epoch_{epoch:03d}_loss_{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=50, verbose=1, mode='min')

result_checker = ResultChecker(model, 10, 500)
model.fit(X, Y, batch_size=batch_size, verbose=1, epochs=n_epoch,
                 callbacks=[result_checker, checkpoint, early_stop])